In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
pd.options.display.float_format = '{:.2f}'.format

In [2]:
# Read in New York Times online csv file with daily COVID data
# Convert FIPS number to integer, convert date to datetime, save file to local csv

us_covid_df = pd.read_csv('https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-counties.csv')
us_covid_df['fips'] = us_covid_df['fips'].fillna(0).astype(np.int64)
us_covid_df['date'] = pd.to_datetime(us_covid_df['date'], yearfirst=True)
us_covid_df.to_csv('us_covid.csv', index=False)

In [3]:
# Read in national population data provided by state, county, FIPS

us_pops_df = pd.read_csv('US_Populations.csv', engine='python', thousands=',')
us_pops_df

,FIPStxt,State,State2,Area_Name,Region,population
0,2000,Alaska,AK,Alaska,State,731545
1,2013,Alaska,AK,Aleutians East Borough,County,3337
2,2016,Alaska,AK,Aleutians West Census Area,County,5634
3,2020,Alaska,AK,Anchorage,County,288000
4,2050,Alaska,AK,Bethel Census Area,County,18386
...,...,...,...,...,...,...
3146,22119,Louisiana,LA,Webster,County,38340
3147,22121,Louisiana,LA,West Baton Rouge,County,26465
3148,22123,Louisiana,LA,West Carroll,County,10830
3149,22125,Louisiana,LA,West Feliciana,County,15568


In [4]:
# Merge New York Times data with population data

us_data_df = pd.merge(us_covid_df, us_pops_df, left_on=['county', 'state'], right_on=['Area_Name', 'State'])
us_data_df

,date,county,state,fips,cases,deaths,FIPStxt,State,State2,Area_Name,Region,population
0,2020-01-21,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
1,2020-01-22,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
2,2020-01-23,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
3,2020-01-24,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
4,2020-01-25,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
...,...,...,...,...,...,...,...,...,...,...,...,...
1013959,2021-02-19,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86
1013960,2021-02-20,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86
1013961,2021-02-21,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86
1013962,2021-02-22,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86


In [5]:
# Separate out data from the state level

states = us_data_df[(us_data_df['Region'] == 'State')]
states

,date,county,state,fips,cases,deaths,FIPStxt,State,State2,Area_Name,Region,population
27547,2020-03-07,District of Columbia,District of Columbia,11001,1,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
27549,2020-03-08,District of Columbia,District of Columbia,11001,1,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
27551,2020-03-09,District of Columbia,District of Columbia,11001,4,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
27553,2020-03-10,District of Columbia,District of Columbia,11001,4,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
27555,2020-03-11,District of Columbia,District of Columbia,11001,10,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
...,...,...,...,...,...,...,...,...,...,...,...,...
628475,2021-02-19,Arkansas,Arkansas,5001,1939,38.00,5000,Arkansas,AR,Arkansas,State,3017804
628477,2021-02-20,Arkansas,Arkansas,5001,1941,38.00,5000,Arkansas,AR,Arkansas,State,3017804
628479,2021-02-21,Arkansas,Arkansas,5001,1942,38.00,5000,Arkansas,AR,Arkansas,State,3017804
628481,2021-02-22,Arkansas,Arkansas,5001,1944,38.00,5000,Arkansas,AR,Arkansas,State,3017804


In [6]:
# Separate out data specific to county level, drop redundant columns
#  Create infection and death rates, updating for each date in the dataframe

counties = us_data_df.drop(['fips', 'State','Area_Name'], axis=1)
counties['current_crate'] = counties['cases']*100000/counties['population']
counties['current_drate'] = counties['deaths']*100000/counties['population']
counties = counties[(counties['Region'] == 'County')]
counties

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate
0,2020-01-21,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
1,2020-01-22,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
2,2020-01-23,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
3,2020-01-24,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
4,2020-01-25,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
...,...,...,...,...,...,...,...,...,...,...,...
1013959,2021-02-19,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00
1013960,2021-02-20,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00
1013961,2021-02-21,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00
1013962,2021-02-22,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00


In [7]:
# Isolate data from most current day.

counties_us = counties[(counties['date'] == '2021-02-23') & (counties['Region'] == 'County')]
counties_us

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate
399,2021-02-23,Snohomish,Washington,30607,530.00,53061,WA,County,822083,3723.10,64.47
796,2021-02-23,Cook,Illinois,470933,9846.00,17031,IL,County,5150233,9143.92,191.18
1192,2021-02-23,Orange,California,259857,3848.00,6059,CA,County,3175692,8182.69,121.17
1587,2021-02-23,Maricopa,Arizona,507043,8910.00,4013,AZ,County,4485414,11304.26,198.64
1982,2021-02-23,Los Angeles,California,1183969,20057.00,6037,CA,County,10039107,11793.57,199.79
...,...,...,...,...,...,...,...,...,...,...,...
1013551,2021-02-23,Wheeler,Oregon,22,1.00,41069,OR,County,1332,1651.65,75.08
1013685,2021-02-23,King,Texas,13,0.00,48269,TX,County,272,4779.41,0.00
1013788,2021-02-23,Esmeralda,Nevada,37,0.00,32009,NV,County,873,4238.26,0.00
1013887,2021-02-23,Loving,Texas,1,0.00,48301,TX,County,169,591.72,0.00


In [8]:
# Create up to date rates so rankings can be performed using the latest data.

counties_us['drate'] = counties_us['deaths']/(counties_us['population']/100000)
counties_us['crate'] = counties_us['cases']/(counties_us['population']/100000)
counties_us = counties_us[(counties_us['Region'] == 'County')]
counties_us

<ipython-input-8-608f6ade14d5>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_us['drate'] = counties_us['deaths']/(counties_us['population']/100000)
<ipython-input-8-608f6ade14d5>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_us['crate'] = counties_us['cases']/(counties_us['population']/100000)


,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate
399,2021-02-23,Snohomish,Washington,30607,530.00,53061,WA,County,822083,3723.10,64.47,64.47,3723.10
796,2021-02-23,Cook,Illinois,470933,9846.00,17031,IL,County,5150233,9143.92,191.18,191.18,9143.92
1192,2021-02-23,Orange,California,259857,3848.00,6059,CA,County,3175692,8182.69,121.17,121.17,8182.69
1587,2021-02-23,Maricopa,Arizona,507043,8910.00,4013,AZ,County,4485414,11304.26,198.64,198.64,11304.26
1982,2021-02-23,Los Angeles,California,1183969,20057.00,6037,CA,County,10039107,11793.57,199.79,199.79,11793.57
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1013551,2021-02-23,Wheeler,Oregon,22,1.00,41069,OR,County,1332,1651.65,75.08,75.08,1651.65
1013685,2021-02-23,King,Texas,13,0.00,48269,TX,County,272,4779.41,0.00,0.00,4779.41
1013788,2021-02-23,Esmeralda,Nevada,37,0.00,32009,NV,County,873,4238.26,0.00,0.00,4238.26
1013887,2021-02-23,Loving,Texas,1,0.00,48301,TX,County,169,591.72,0.00,0.00,591.72


In [9]:
# Create ranking of infection rates by county for each state

counties_us.sort_values(by=['state', 'crate', 'county'], inplace=True, ascending=(True, False, True))
counties_us['state_case']=counties_us.groupby([counties_us.State2]).cumcount()+1
counties_us

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_case
714877,2021-02-23,Clarke,Alabama,3423,48.00,1025,AL,County,23622,14490.73,203.20,203.20,14490.73,1
657032,2021-02-23,Hale,Alabama,2080,64.00,1065,AL,County,14651,14196.98,436.83,436.83,14196.98,2
514581,2021-02-23,Lowndes,Alabama,1301,48.00,1085,AL,County,9726,13376.52,493.52,493.52,13376.52,3
514916,2021-02-23,Marengo,Alabama,2434,54.00,1091,AL,County,18863,12903.57,286.27,286.27,12903.57,4
456459,2021-02-23,Etowah,Alabama,13091,309.00,1055,AL,County,102268,12800.68,302.15,302.15,12800.68,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
964410,2021-02-23,Platte,Wyoming,583,11.00,56031,WY,County,8393,6946.26,131.06,131.06,6946.26,19
627487,2021-02-23,Sublette,Wyoming,671,7.00,56035,WY,County,9831,6825.35,71.20,71.20,6825.35,20
866135,2021-02-23,Niobrara,Wyoming,148,2.00,56027,WY,County,2356,6281.83,84.89,84.89,6281.83,21
778839,2021-02-23,Lincoln,Wyoming,1213,11.00,56023,WY,County,19830,6116.99,55.47,55.47,6116.99,22


In [10]:
# Create ranking of death rates by county for each state

counties_us.sort_values(by=['state', 'drate', 'county'], inplace=True, ascending=(True, False, True))
counties_us['state_death']=counties_us.groupby([counties_us.State2]).cumcount()+1
counties_us

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_case,state_death
514581,2021-02-23,Lowndes,Alabama,1301,48.00,1085,AL,County,9726,13376.52,493.52,493.52,13376.52,3,1
657032,2021-02-23,Hale,Alabama,2080,64.00,1065,AL,County,14651,14196.98,436.83,436.83,14196.98,2,2
455115,2021-02-23,Clay,Alabama,1425,54.00,1027,AL,County,13235,10766.91,408.01,408.01,10766.91,22,3
554778,2021-02-23,Greene,Alabama,867,32.00,1063,AL,County,8111,10689.19,394.53,394.53,10689.19,23,4
219436,2021-02-23,Walker,Alabama,6466,240.00,1127,AL,County,63521,10179.31,377.83,377.83,10179.31,32,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
553441,2021-02-23,Hot Springs,Wyoming,360,3.00,56017,WY,County,4413,8157.72,67.98,67.98,8157.72,16,19
759546,2021-02-23,Uinta,Wyoming,2036,12.00,56041,WY,County,20226,10066.25,59.33,59.33,10066.25,4,20
778839,2021-02-23,Lincoln,Wyoming,1213,11.00,56023,WY,County,19830,6116.99,55.47,55.47,6116.99,22,21
218410,2021-02-23,Teton,Wyoming,3333,9.00,56039,WY,County,23464,14204.74,38.36,38.36,14204.74,1,22


In [11]:
# Merge data from today with historical data to provide current rates + rates from previous points in time.

usa = pd.merge(counties, counties_us[['state', 'county','drate', 'crate', 'state_death', 'state_case']], on=['state', 'county'])
usa

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case
0,2020-01-21,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,64.47,3723.10,14,21
1,2020-01-22,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,64.47,3723.10,14,21
2,2020-01-23,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,64.47,3723.10,14,21
3,2020-01-24,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,64.47,3723.10,14,21
4,2020-01-25,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,64.47,3723.10,14,21
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1011565,2021-02-19,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,3
1011566,2021-02-20,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,3
1011567,2021-02-21,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,3
1011568,2021-02-22,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,3


In [12]:
# Break out daily case data from the New York Times csv that is cumulative in nature

usa.sort_values(by=['state', 'state_case', 'date'], inplace=True)
usa['case_day'] = usa.groupby(['state','county']).cases.diff().fillna(usa.cases)
usa

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day
712155,2020-04-01,Clarke,Alabama,2,0.00,1025,AL,County,23622,8.47,0.00,203.20,14490.73,46,1,2.00
712156,2020-04-02,Clarke,Alabama,2,0.00,1025,AL,County,23622,8.47,0.00,203.20,14490.73,46,1,0.00
712157,2020-04-03,Clarke,Alabama,3,0.00,1025,AL,County,23622,12.70,0.00,203.20,14490.73,46,1,1.00
712158,2020-04-04,Clarke,Alabama,7,0.00,1025,AL,County,23622,29.63,0.00,203.20,14490.73,46,1,4.00
712159,2020-04-05,Clarke,Alabama,9,0.00,1025,AL,County,23622,38.10,0.00,203.20,14490.73,46,1,2.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
791763,2021-02-19,Crook,Wyoming,417,11.00,56011,WY,County,7584,5498.42,145.04,145.04,5498.42,6,23,0.00
791764,2021-02-20,Crook,Wyoming,417,11.00,56011,WY,County,7584,5498.42,145.04,145.04,5498.42,6,23,0.00
791765,2021-02-21,Crook,Wyoming,417,11.00,56011,WY,County,7584,5498.42,145.04,145.04,5498.42,6,23,0.00
791766,2021-02-22,Crook,Wyoming,417,11.00,56011,WY,County,7584,5498.42,145.04,145.04,5498.42,6,23,0.00


In [13]:
# Break out daily death data from the New York Times csv that is cumulative in nature

usa.sort_values(by=['state', 'state_death', 'date'], inplace=True)
usa['death_day'] = usa.groupby(['state','county']).deaths.diff().fillna(usa.deaths)
usa

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
512519,2020-03-26,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,493.52,13376.52,1,3,1.00,0.00
512520,2020-03-27,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,493.52,13376.52,1,3,0.00,0.00
512521,2020-03-28,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,493.52,13376.52,1,3,0.00,0.00
512522,2020-03-29,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,493.52,13376.52,1,3,0.00,0.00
512523,2020-03-30,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,493.52,13376.52,1,3,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
510839,2021-02-19,Albany,Wyoming,3880,11.00,56001,WY,County,38880,9979.42,28.29,28.29,10010.29,23,6,9.00,0.00
510840,2021-02-20,Albany,Wyoming,3880,11.00,56001,WY,County,38880,9979.42,28.29,28.29,10010.29,23,6,0.00,0.00
510841,2021-02-21,Albany,Wyoming,3887,11.00,56001,WY,County,38880,9997.43,28.29,28.29,10010.29,23,6,7.00,0.00
510842,2021-02-22,Albany,Wyoming,3893,11.00,56001,WY,County,38880,10012.86,28.29,28.29,10010.29,23,6,6.00,0.00


In [14]:
# Export updated national data to csv for use in Tableau

usa.to_csv('daily_covid_us.csv', index=False)

In [15]:
# Filter California data to show only numbers for today

counties_cal = usa[(usa['date'] == '2021-02-23') & (usa['state'] == 'California')]
counties_cal

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
261812,2021-02-23,Imperial,California,26840,585.00,6025,CA,County,181215,14811.14,322.82,322.82,14811.14,1,2,0.00,2.00
1982,2021-02-23,Los Angeles,California,1183969,20057.00,6037,CA,County,10039107,11793.57,199.79,199.79,11793.57,2,5,1975.00,153.00
457755,2021-02-23,Inyo,California,1248,34.00,6027,CA,County,18039,6918.34,188.48,188.48,6918.34,3,28,16.00,0.00
58537,2021-02-23,Stanislaus,California,55487,937.00,6099,CA,County,550660,10076.45,170.16,170.16,10076.45,4,10,140.00,6.00
72534,2021-02-23,Tulare,California,47680,731.00,6107,CA,County,466195,10227.48,156.80,156.80,10227.48,5,9,118.00,9.00
27192,2021-02-23,Riverside,California,288541,3695.00,6065,CA,County,2470546,11679.24,149.56,149.56,11679.24,6,6,719.00,31.00
554720,2021-02-23,Kings,California,21916,218.00,6031,CA,County,152940,14329.80,142.54,142.54,14329.80,7,3,35.00,2.00
414973,2021-02-23,Merced,California,28749,393.00,6047,CA,County,277680,10353.28,141.53,141.53,10353.28,8,8,94.00,2.00
26484,2021-02-23,Fresno,California,94584,1397.00,6019,CA,County,999101,9466.91,139.83,139.83,9466.91,9,14,210.00,30.00
46605,2021-02-23,San Joaquin,California,66290,1042.00,6077,CA,County,762148,8697.79,136.72,136.72,8697.79,10,18,244.00,3.00


In [16]:
# Show top 10 California counties based on largest current infection rate

counties_cal.sort_values(by=['crate'], inplace=True, ascending=False)
top10_ca_case_rates = counties_cal.head(10)
top10_ca_case_rates

<ipython-input-16-94608615179b>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_cal.sort_values(by=['crate'], inplace=True, ascending=False)


,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
963961,2021-02-23,Lassen,California,5594,19.00,6035,CA,County,30573,18297.19,62.15,62.15,18297.19,36,1,3.00,0.00
261812,2021-02-23,Imperial,California,26840,585.00,6025,CA,County,181215,14811.14,322.82,322.82,14811.14,1,2,0.00,2.00
554720,2021-02-23,Kings,California,21916,218.00,6031,CA,County,152940,14329.80,142.54,142.54,14329.80,7,3,35.00,2.00
130626,2021-02-23,San Bernardino,California,285081,2651.00,6071,CA,County,2180085,13076.60,121.60,121.60,13076.60,12,4,215.00,14.00
1982,2021-02-23,Los Angeles,California,1183969,20057.00,6037,CA,County,10039107,11793.57,199.79,199.79,11793.57,2,5,1975.00,153.00
27192,2021-02-23,Riverside,California,288541,3695.00,6065,CA,County,2470546,11679.24,149.56,149.56,11679.24,6,6,719.00,31.00
164133,2021-02-23,Kern,California,102137,786.00,6029,CA,County,900202,11346.01,87.31,87.31,11346.01,24,7,288.00,3.00
414973,2021-02-23,Merced,California,28749,393.00,6047,CA,County,277680,10353.28,141.53,141.53,10353.28,8,8,94.00,2.00
72534,2021-02-23,Tulare,California,47680,731.00,6107,CA,County,466195,10227.48,156.80,156.80,10227.48,5,9,118.00,9.00
58537,2021-02-23,Stanislaus,California,55487,937.00,6099,CA,County,550660,10076.45,170.16,170.16,10076.45,4,10,140.00,6.00


In [17]:
# Show top 10 California counties based on largest current death rate

counties_cal.sort_values(by=['drate'], inplace=True, ascending=False)
top10_ca_death_rates = counties_cal.head(10)
top10_ca_death_rates

<ipython-input-17-018d0b919d54>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_cal.sort_values(by=['drate'], inplace=True, ascending=False)


,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
261812,2021-02-23,Imperial,California,26840,585.00,6025,CA,County,181215,14811.14,322.82,322.82,14811.14,1,2,0.00,2.00
1982,2021-02-23,Los Angeles,California,1183969,20057.00,6037,CA,County,10039107,11793.57,199.79,199.79,11793.57,2,5,1975.00,153.00
457755,2021-02-23,Inyo,California,1248,34.00,6027,CA,County,18039,6918.34,188.48,188.48,6918.34,3,28,16.00,0.00
58537,2021-02-23,Stanislaus,California,55487,937.00,6099,CA,County,550660,10076.45,170.16,170.16,10076.45,4,10,140.00,6.00
72534,2021-02-23,Tulare,California,47680,731.00,6107,CA,County,466195,10227.48,156.80,156.80,10227.48,5,9,118.00,9.00
27192,2021-02-23,Riverside,California,288541,3695.00,6065,CA,County,2470546,11679.24,149.56,149.56,11679.24,6,6,719.00,31.00
554720,2021-02-23,Kings,California,21916,218.00,6031,CA,County,152940,14329.80,142.54,142.54,14329.80,7,3,35.00,2.00
414973,2021-02-23,Merced,California,28749,393.00,6047,CA,County,277680,10353.28,141.53,141.53,10353.28,8,8,94.00,2.00
26484,2021-02-23,Fresno,California,94584,1397.00,6019,CA,County,999101,9466.91,139.83,139.83,9466.91,9,14,210.00,30.00
46605,2021-02-23,San Joaquin,California,66290,1042.00,6077,CA,County,762148,8697.79,136.72,136.72,8697.79,10,18,244.00,3.00


In [18]:
# Create dataframe with historical data for Top 10 counties based on current case rate

top10c_df = top10_ca_case_rates[['county', 'state']]
top10_ca_case_df = pd.merge(usa, top10c_df, on=['county', 'state'])
top10_ca_case_df['date'] = pd.to_datetime(top10_ca_case_df['date']).dt.strftime('%m/%d/%y')
top10_ca_case_df = top10_ca_case_df[['date', 'county','cases', 'deaths', 'current_crate', 'current_drate', 'drate', 'crate', 'state_death', 'state_case', 'case_day', 'death_day']]
top10_ca_case_df

,date,county,cases,deaths,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
0,03/20/20,Imperial,4,0.00,2.21,0.00,322.82,14811.14,1,2,4.00,0.00
1,03/21/20,Imperial,4,0.00,2.21,0.00,322.82,14811.14,1,2,0.00,0.00
2,03/22/20,Imperial,4,0.00,2.21,0.00,322.82,14811.14,1,2,0.00,0.00
3,03/23/20,Imperial,4,0.00,2.21,0.00,322.82,14811.14,1,2,0.00,0.00
4,03/24/20,Imperial,9,0.00,4.97,0.00,322.82,14811.14,1,2,5.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...
3422,02/19/21,Lassen,5558,19.00,18179.44,62.15,62.15,18297.19,36,1,0.00,0.00
3423,02/20/21,Lassen,5558,19.00,18179.44,62.15,62.15,18297.19,36,1,0.00,0.00
3424,02/21/21,Lassen,5558,19.00,18179.44,62.15,62.15,18297.19,36,1,0.00,0.00
3425,02/22/21,Lassen,5591,19.00,18287.38,62.15,62.15,18297.19,36,1,33.00,0.00


In [19]:
# Create dataframe with historical data for Top 10 counties based on current death rate

top10d_df = top10_ca_death_rates[['county', 'state']]
top10_ca_death_df = pd.merge(usa, top10d_df, on=['county', 'state'])
top10_ca_death_df['date'] = pd.to_datetime(top10_ca_death_df['date']).dt.strftime('%m/%d/%y')
top10_ca_death_df = top10_ca_death_df[['date', 'county','cases', 'deaths', 'current_crate', 'current_drate', 'drate', 'crate', 'state_death', 'state_case', 'case_day', 'death_day']]
top10_ca_death_df

,date,county,cases,deaths,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
0,03/20/20,Imperial,4,0.00,2.21,0.00,322.82,14811.14,1,2,4.00,0.00
1,03/21/20,Imperial,4,0.00,2.21,0.00,322.82,14811.14,1,2,0.00,0.00
2,03/22/20,Imperial,4,0.00,2.21,0.00,322.82,14811.14,1,2,0.00,0.00
3,03/23/20,Imperial,4,0.00,2.21,0.00,322.82,14811.14,1,2,0.00,0.00
4,03/24/20,Imperial,9,0.00,4.97,0.00,322.82,14811.14,1,2,5.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...
3496,02/19/21,San Joaquin,65891,1026.00,8645.43,134.62,136.72,8697.79,10,18,186.00,5.00
3497,02/20/21,San Joaquin,65891,1026.00,8645.43,134.62,136.72,8697.79,10,18,0.00,0.00
3498,02/21/21,San Joaquin,65891,1026.00,8645.43,134.62,136.72,8697.79,10,18,0.00,0.00
3499,02/22/21,San Joaquin,66046,1039.00,8665.77,136.33,136.72,8697.79,10,18,155.00,13.00


In [20]:
# Export California Top 10 case data to csv

top10_ca_case_df.to_csv('top10cases.csv', float_format='%.2f', index=False)

In [21]:
# Export California Top 10 death data to csv

top10_ca_death_df.to_csv('top10deaths.csv', float_format='%.2f', index=False)